# Etapa 1 (Landing & Bronze)
Aquisição de Dados e Armazenamento de Dados em PostgreSQL.
- Baixe o conjunto de dados "Inside Airbnb" do Rio de Janeiro da fonte oficial (http://insideairbnb.com/) e promova uma estruturação simples nos dados.
- Crie um banco de dados PostgreSQL para armazenar os dados brutos das 3 tabelas ("Listing", "Reviews" e Calendar") na camada "bronze".

## Landing

In [ ]:
from modules.etl import Landing

url = [
    'https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/listings.csv.gz',
    'https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/calendar.csv.gz',
    'https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/reviews.csv.gz'
]
landing_step = Landing(landing_path='data/landing')
landing_step.extract(url=url)
landing_step.transform()

## Bronze

In [ ]:
import os
import pandas as pd
from modules.utils import clean_path
from modules.database import PostgreSQL

## Criação de base de dados
database = PostgreSQL() # Inicializando a classe com as variaveis de conexão com o banco
database.create_database('datalake') # Criando o database datalake

## Criação dos Schema
try:
    print("Iniciando a criação dos schemas bronze, silver e gold")

    database.create_schema(schema_name='bronze',database='datalake')
    database.create_schema(schema_name='silver',database='datalake')
    database.create_schema(schema_name='gold',database='datalake')

    print("Fim da criação dos schemas")
except Exception as e:
    print(f"Erro ao criar os schemas: {e}")
    raise e

## Inserção dos dataframes
try:
    print("Inicio do carregamento de dados (Bronze)")

    data_path = './data/landing' # Define onde estão os CSVs
    df_listings = pd.read_csv(os.path.join(data_path,'listings.csv')) # Lê o csv e transforma em um dataframe Python
    database.insert_dataframe(dataframe=df_listings,table_name='listings',schema='bronze',database='datalake',if_exists='append') # Insere o Dataframe no banco de dados
    del df_listings # Libera o espaço que a variável consome na memória RAM

    df_reviews = pd.read_csv(os.path.join(data_path,'reviews.csv')) # Lê o csv e transforma em um dataframe Python
    database.insert_dataframe(dataframe=df_reviews,table_name='reviews',schema='bronze',database='datalake',if_exists='append') # Insere o Dataframe no banco de dados
    del df_reviews # Libera o espaço que a variável consome na memória RAM

    df_calendar = pd.read_csv(os.path.join(data_path,'calendar.csv')) # Lê o csv e transforma em um dataframe Python
    database.insert_dataframe(dataframe=df_calendar,table_name='calendar',schema='bronze',database='datalake',if_exists='append') # Insere o Dataframe no banco de dados
    del df_calendar # Libera o espaço que a variável consome na memória RAM

    clean_path(path='data/landing') # Limpando os arquivos CSVs

    print("Fim do carregamento de dados (Bronze)")
except Exception as e:
    print(f"Erro ao inserir os dados: {e}")
    raise e

# Etapa 2 (Silver)
**Data Clean - Camada Silver**

- Identifique e lide com valores ausentes, duplicatas e outliers nos dados brutos da camada "bronze".
- Padronize e limpe os nomes das colunas, convertendo-os em um formato consistente.
- Realize uma limpeza textual em campos, como descrições de propriedades, removendo caracteres especiais e erros de digitação.

## Listings

In [1]:
import pandas as pd
from modules.database import PostgreSQL

# Importanddo a tabela
database = 'datalake'
df_silver_listings = pd.read_sql_table(table_name='listings',schema='bronze',con=PostgreSQL().create_connection(database=database),index_col='index')

# Alterar tipo de coluna para datetime64[ns] para colunas: 'last_scraped', 'host_since' e 3 outras colunas
df_silver_listings = df_silver_listings.astype({'last_scraped': 'datetime64[ns]', 'host_since': 'datetime64[ns]', 'calendar_last_scraped': 'datetime64[ns]', 'last_review': 'datetime64[ns]', 'first_review': 'datetime64[ns]'})

# Alterar tipo de coluna para category para coluna: 'source'
df_silver_listings = df_silver_listings.astype({'source': 'category'})

# Removendo caracteres especiais das colunas 'neighborhood_overview' e 'host_about'
import re
df_silver_listings['neighborhood_overview'] = df_silver_listings['neighborhood_overview'].apply(lambda x: None if x is None else re.sub(r'[^a-zA-Z0-9\s.()]','',x))
df_silver_listings['host_about'] = df_silver_listings['host_about'].apply(lambda x: None if x is None else re.sub(r'[^a-zA-Z0-9\s.()]','',x))

# Remover colunas 100% vazias
df_silver_listings = df_silver_listings.dropna(axis=1,how='all')

# Removendo linhas duplicadas
df_silver_listings = df_silver_listings.drop_duplicates()

# Substituir todas as instâncias de "t" por "1" nas colunas: 'host_is_superhost', 'host_has_profile_pic' e 3 outras colunas
colunas_a_ser_tratadas = ['host_is_superhost','host_has_profile_pic','host_identity_verified','instant_bookable','has_availability']
for coluna in colunas_a_ser_tratadas:
    df_silver_listings[coluna] = df_silver_listings[coluna].apply(lambda x: 1 if str(x).lower() == 't' else 0)

# Alterar tipo de coluna para category para coluna: 'host_response_time'
df_silver_listings = df_silver_listings.astype({'host_response_time': 'category'})

# Convertendo as colunas host_response_rate e host_acceptance_rate em int
colunas_a_serem_convertidas = ['host_response_rate','host_acceptance_rate']
for coluna in colunas_a_serem_convertidas:
    df_silver_listings[coluna] = df_silver_listings[coluna].apply(lambda x: 0 if x is None else int(x.replace("%",'')))

# Tratando a coluna bathrooms_text
df_silver_listings['bathrooms_text'] = df_silver_listings['bathrooms_text'].fillna('0 bath') # Tratando vazios
df_silver_listings['num_of_bathrooms'] = df_silver_listings['bathrooms_text'].str.extract(r"(\d+(?:\.\d+)?)", expand=False).astype(float).fillna(0.0) # Retirando os valores númericos da coluna
df_silver_listings['type_of_bathroom'] = df_silver_listings['bathrooms_text'].str.replace(r'(\d+\.?\d*)', '', regex=True) # Defina o tipo de banheiro
df_silver_listings['type_of_bathroom'] = df_silver_listings['type_of_bathroom'].str.replace(r's?$', '', regex=True).apply(lambda x: x.strip()) # Remova 's' do final de 'baths e traz
df_silver_listings = df_silver_listings.astype({'type_of_bathroom':'category'}) # Mudando o tipo da coluna type_of_bathroom
df_silver_listings.drop(columns='bathrooms_text',inplace=True) # Dropando a coluna de origem

# Substituir valores ausentes por 0 nas colunas: 'review_scores_rating', 'review_scores_accuracy' e 6 outras colunas
df_silver_listings = df_silver_listings.fillna({'review_scores_rating': 0, 'review_scores_accuracy': 0, 'review_scores_cleanliness': 0, 'review_scores_checkin': 0, 'review_scores_communication': 0, 'review_scores_location': 0, 'review_scores_value': 0, 'reviews_per_month': 0})

## Reviews

In [9]:
import pandas as pd
from modules.database import PostgreSQL

# Importanddo a tabela
database = 'datalake'
df_silver_reviews = pd.read_sql_table(table_name='reviews',schema='bronze',con=PostgreSQL().create_connection(database=database),index_col='index')

# Alterar tipo de coluna para datetime64[ns] para coluna: 'date'
df_silver_reviews = df_silver_reviews.astype({'date': 'datetime64[ns]'})

# Substituir todas as instâncias de "<br/>" por "\n" na coluna: 'comments'
df_silver_reviews['comments'] = df_silver_reviews['comments'].str.replace("<br/>", "\n", case=False, regex=False)

# Remover espaço em branco à direita e à esquerda na coluna: 'reviewer_name'
df_silver_reviews['reviewer_name'] = df_silver_reviews['reviewer_name'].str.strip()

# Remover linhas duplicadas em todas as colunas
df_silver_reviews = df_silver_reviews.drop_duplicates()

## Calendar

In [10]:
import pandas as pd
from modules.database import PostgreSQL

# Importanddo a tabela
database = 'datalake'
df_silver_calendar = pd.read_sql_table(table_name='calendar',schema='bronze',con=PostgreSQL().create_connection(database=database),index_col='index')

# Etapa 3 (Silver)
**Data Quality - Camada Silver**

- Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".
- Implemente verificações para garantir que os dados da camada "silver" estejam em conformidade com essas métricas.
- Estabeleça um sistema de monitoramento contínuo da qualidade dos dados da camada "silver".

# Etapa 4 (Silver)
**Testes de Qualidade - Camada Silver**

- Utilize a biblioteca Great Expectations para criar testes de qualidade automatizados que verifiquem as expectativas definidas para os dados da camada "silver".
- Desenvolva testes que assegurem que os dados da camada "silver" atendam às regras de negócios e aos requisitos de qualidade.

## Listing

In [2]:
import modules.utils as utils
import great_expectations as gx

gx_df_listings = gx.from_pandas(df_silver_listings)

# Verificando os tipos das colunas
print("# COLUNAS DATEIME")
colunas_datetime = ['last_scraped', 'host_since', 'calendar_last_scraped', 'last_review', 'first_review']
utils.verifica_colunas_datetime(gx_df_listings, colunas_datetime)

# Checando os valores Categóricos
print("\n# COLUNAS CATEGORICAS")
coluna_a_ser_analisada, valores_esperados = 'source', ['city scrape', 'previous scrape']
utils.verificar_colunas_categoricas(gx_df_listings,coluna_a_ser_analisada, valores_esperados)

coluna_a_ser_analisada, valores_esperados = 'room_type', ['Entire home/apt', 'Private room', 'Shared room', 'Hotel room']
utils.verificar_colunas_categoricas(gx_df_listings,coluna_a_ser_analisada, valores_esperados)

# Checando as colunas Booleanas
print("\n# COLUNAS BOOLEANAS")
colunas_booleanas = ['host_is_superhost','host_has_profile_pic','host_identity_verified','instant_bookable','has_availability']
utils.verificar_colunas_booleanas(gx_df=gx_df_listings,list_of_columns=colunas_booleanas)

# Verificando nulos
print("\n# COLUNAS NÃO VAZIAS")
colunas_nao_vazias = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']
utils.verificar_collunas_com_none(gx_df=gx_df_listings,list_of_columns=colunas_nao_vazias)


# COLUNAS DATEIME
A coluna 'last_scraped' é válida. (Tipo e Valor)
A coluna 'host_since' é válida. (Tipo e Valor)
A coluna 'calendar_last_scraped' é válida. (Tipo e Valor)
A coluna 'last_review' é válida. (Tipo e Valor)
A coluna 'first_review' é válida. (Tipo e Valor)

# COLUNAS CATEGORICAS
Os valores da coluna source, só contém valores esperados. (['city scrape', 'previous scrape'])
Os valores da coluna room_type, só contém valores esperados. (['Entire home/apt', 'Private room', 'Shared room', 'Hotel room'])

# COLUNAS BOOLEANAS
A coluna 'host_is_superhost' é válida. (Tipo e Valor)
A coluna 'host_has_profile_pic' é válida. (Tipo e Valor)
A coluna 'host_identity_verified' é válida. (Tipo e Valor)
A coluna 'instant_bookable' é válida. (Tipo e Valor)
A coluna 'has_availability' é válida. (Tipo e Valor)

# COLUNAS NÃO VAZIAS
A coluna 'review_scores_rating' é válida. (Não tem valores nulos)
A coluna 'review_scores_accuracy' é válida. (Não tem valores nulos)
A coluna 'review_scores_cleanlin